# Exploratory Data Analysis - clustering

Попробуем кластеризовать.

In [ ]:
# Re-install pytables for hdf support (google colab has old & glitchy pytables)
#!pip uninstall -y tables
#!pip install tables

In [1]:
import pandas as pd
df = pd.read_hdf('fe1.h5')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227654 entries, 0 to 467481
Data columns (total 3 columns):
filename    227654 non-null object
text        227654 non-null object
lang        227654 non-null object
dtypes: object(3)
memory usage: 6.9+ MB


In [2]:
df.head()

,filename,text,lang
0,dataset/20191108/15/1491978796208217930.html,\n\n\n\n\n\n\n\n\n\n\n\nNewcastle United v Bou...,en
4,dataset/20191108/15/6359253557838878602.html,\n\n\n\n\n\n\n\n\n\n\n\nМинкомсвязь негативно ...,ru
6,dataset/20191108/15/8549607152500932485.html,\n\n\n\n\n\n\n\n\n\n\n\nTether Responds to Mar...,en
7,dataset/20191108/15/3752801377276103486.html,\n\n\n\n\n\n\n\n\n\n\n\nAnger over Las Vegas b...,en
10,dataset/20191108/15/5545714965965021212.html,\n\n\n\n\n\n\n\n\n\n\n\nAmerican Truck Simulat...,en


In [ ]:
#!pip install nltk

In [3]:
import nltk
#nltk.download('stopwords', download_dir='nltk_data')
nltk.data.path.append('nltk_data')

In [4]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=stopwords.words('english'))
data = cv.fit_transform(df[df['lang'] == 'en']['text'].values)

CPU times: user 40.5 s, sys: 1.71 s, total: 42.2 s
Wall time: 42.3 s


Коичество признаков:

In [6]:
data.shape[1]

340401

Для начала применим алгоритм K-Means. Нужно найти подход для выбора количества кластеров.

In [8]:
%%time
from sklearn.cluster import MiniBatchKMeans
cl = MiniBatchKMeans(n_clusters=10)
cl.fit(data)
cl.inertia_

CPU times: user 8.04 s, sys: 9.41 s, total: 17.4 s
Wall time: 3.56 s


126343830.33273417

In [11]:
%%time
cl = MiniBatchKMeans(n_clusters=1000)
cl.fit(data)
cl.inertia_

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=1000. Setting it to 3*k
  init_size=init_size)


CPU times: user 13min 7s, sys: 7min 43s, total: 20min 50s
Wall time: 7min 9s


116025605.704954

In [12]:
%%time
cl = MiniBatchKMeans(n_clusters=data.shape[0])
cl.fit(data)
cl.inertia_

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=107186. Setting it to 3*k
  init_size=init_size)


MemoryError: Unable to allocate array with shape (107186, 340401) and data type float64

Не хавтает памяти.